In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

X_train = pd.read_json("../input/train.json")
X_test = pd.read_json("../input/test.json")


In [ ]:
X_train.head()

In [ ]:

X_train["longitude"]


In [ ]:
X_train["latitude"]

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
sample=pd.read_csv("../input/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
print(check_output(["ls", "../input/images_sample/"]).decode("utf8"))



In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import random
from math import exp
import xgboost as xgb

random.seed(321)
np.random.seed(321)

X_train = pd.read_json("../input/train.json")
X_test = pd.read_json("../input/test.json")

interest_level_map = {'low': 0, 'medium': 1, 'high': 2}
X_train['interest_level'] = X_train['interest_level'].apply(lambda x: interest_level_map[x])
X_test['interest_level'] = -1

#add features
feature_transform = CountVectorizer(stop_words='english', max_features=150)
X_train['features'] = X_train["features"].apply(lambda x: " ".join(["_".join(i.lower().split(" ")) for i in x]))
X_test['features'] = X_test["features"].apply(lambda x: " ".join(["_".join(i.lower().split(" ")) for i in x]))
feature_transform.fit(list(X_train['features']) + list(X_test['features']))

train_size = len(X_train)
low_count = len(X_train[X_train['interest_level'] == 0])
medium_count = len(X_train[X_train['interest_level'] == 1])
high_count = len(X_train[X_train['interest_level'] == 2])

def find_objects_with_only_one_record(feature_name):
    temp = pd.concat([X_train[feature_name].reset_index(), 
                      X_test[feature_name].reset_index()])
    temp = temp.groupby(feature_name, as_index = False).count()
    return temp[temp['index'] == 1]

managers_with_one_lot = find_objects_with_only_one_record('manager_id')
buildings_with_one_lot = find_objects_with_only_one_record('building_id')
addresses_with_one_lot = find_objects_with_only_one_record('display_address')

lambda_val = None
k=5.0
f=1.0
r_k=0.01 
g = 1.0

def categorical_average(variable, y, pred_0, feature_name):
    def calculate_average(sub1, sub2):
        s = pd.DataFrame(data = {
                                 variable: sub1.groupby(variable, as_index = False).count()[variable],                              
                                 'sumy': sub1.groupby(variable, as_index = False).sum()['y'],
                                 'avgY': sub1.groupby(variable, as_index = False).mean()['y'],
                                 'cnt': sub1.groupby(variable, as_index = False).count()['y']
                                 })
                                 
        tmp = sub2.merge(s.reset_index(), how='left', left_on=variable, right_on=variable) 
        del tmp['index']                       
        tmp.loc[pd.isnull(tmp['cnt']), 'cnt'] = 0.0
        tmp.loc[pd.isnull(tmp['cnt']), 'sumy'] = 0.0

        def compute_beta(row):
            cnt = row['cnt'] if row['cnt'] < 200 else float('inf')
            return 1.0 / (g + exp((cnt - k) / f))
            
        if lambda_val is not None:
            tmp['beta'] = lambda_val
        else:
            tmp['beta'] = tmp.apply(compute_beta, axis = 1)
            
        tmp['adj_avg'] = tmp.apply(lambda row: (1.0 - row['beta']) * row['avgY'] + row['beta'] * row['pred_0'],
                                   axis = 1)
                                   
        tmp.loc[pd.isnull(tmp['avgY']), 'avgY'] = tmp.loc[pd.isnull(tmp['avgY']), 'pred_0']
        tmp.loc[pd.isnull(tmp['adj_avg']), 'adj_avg'] = tmp.loc[pd.isnull(tmp['adj_avg']), 'pred_0']
        tmp['random'] = np.random.uniform(size = len(tmp))
        tmp['adj_avg'] = tmp.apply(lambda row: row['adj_avg'] *(1 + (row['random'] - 0.5) * r_k),
                                   axis = 1)
    
        return tmp['adj_avg'].ravel()
     
    #cv for training set 
    k_fold = StratifiedKFold(5)
    X_train[feature_name] = -999 
    for (train_index, cv_index) in k_fold.split(np.zeros(len(X_train)),
                                                X_train['interest_level'].ravel()):
        sub = pd.DataFrame(data = {variable: X_train[variable],
                                   'y': X_train[y],
                                   'pred_0': X_train[pred_0]})
            
        sub1 = sub.iloc[train_index]        
        sub2 = sub.iloc[cv_index]
        
        X_train.loc[cv_index, feature_name] = calculate_average(sub1, sub2)
    
    #for test set
    sub1 = pd.DataFrame(data = {variable: X_train[variable],
                                'y': X_train[y],
                                'pred_0': X_train[pred_0]})
    sub2 = pd.DataFrame(data = {variable: X_test[variable],
                                'y': X_test[y],
                                'pred_0': X_test[pred_0]})
    X_test.loc[:, feature_name] = calculate_average(sub1, sub2)                               

def transform_data(X):
    #add features    
    feat_sparse = feature_transform.transform(X["features"])
    vocabulary = feature_transform.vocabulary_
    del X['features']
    X1 = pd.DataFrame([ pd.Series(feat_sparse[i].toarray().ravel()) for i in np.arange(feat_sparse.shape[0]) ])
    X1.columns = list(sorted(vocabulary.keys()))
    X = pd.concat([X.reset_index(), X1.reset_index()], axis = 1)
    del X['index']
    
    X["num_photos"] = X["photos"].apply(len)
    X['created'] = pd.to_datetime(X["created"])
    X["num_description_words"] = X["description"].apply(lambda x: len(x.split(" ")))
    X['price_per_bed'] = X['price'] / X['bedrooms']    
    X['price_per_bath'] = X['price'] / X['bathrooms']
    X['price_per_room'] = X['price'] / (X['bathrooms'] + X['bedrooms'] )
    
    X['low'] = 0
    X.loc[X['interest_level'] == 0, 'low'] = 1
    X['medium'] = 0
    X.loc[X['interest_level'] == 1, 'medium'] = 1
    X['high'] = 0
    X.loc[X['interest_level'] == 2, 'high'] = 1
    
    X['display_address'] = X['display_address'].apply(lambda x: x.lower().strip())
    X['street_address'] = X['street_address'].apply(lambda x: x.lower().strip())
    
    X['pred0_low'] = low_count * 1.0 / train_size
    X['pred0_medium'] = medium_count * 1.0 / train_size
    X['pred0_high'] = high_count * 1.0 / train_size
    
    X.loc[X['manager_id'].isin(managers_with_one_lot['manager_id'].ravel()), 
          'manager_id'] = "-1"
    X.loc[X['building_id'].isin(buildings_with_one_lot['building_id'].ravel()), 
          'building_id'] = "-1"
    X.loc[X['display_address'].isin(addresses_with_one_lot['display_address'].ravel()), 
          'display_address'] = "-1"
          
    return X

def normalize_high_cordiality_data():
    high_cardinality = ["building_id", "manager_id"]
    for c in high_cardinality:
        categorical_average(c, "medium", "pred0_medium", c + "_mean_medium")
        categorical_average(c, "high", "pred0_high", c + "_mean_high")

def transform_categorical_data():
    categorical = ['building_id', 'manager_id', 
                   'display_address', 'street_address']
                   
    for f in categorical:
        encoder = LabelEncoder()
        encoder.fit(list(X_train[f]) + list(X_test[f])) 
        X_train[f] = encoder.transform(X_train[f].ravel())
        X_test[f] = encoder.transform(X_test[f].ravel())
                  

def remove_columns(X):
    columns = ["photos", "pred0_high", "pred0_low", "pred0_medium",
               "description", "low", "medium", "high",
               "interest_level", "created"]
    for c in columns:
        del X[c]

print("Starting transformations")        
X_train = transform_data(X_train)    
X_test = transform_data(X_test) 
y = X_train['interest_level'].ravel()

print("Normalizing high cordiality data...")
normalize_high_cordiality_data()
transform_categorical_data()

remove_columns(X_train)
remove_columns(X_test)

print("Start fitting...")

param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.02
param['max_depth'] = 4
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 321
param['nthread'] = 8
num_rounds = 2000

xgtrain = xgb.DMatrix(X_train, label=y)
clf = xgb.train(param, xgtrain, num_rounds)

print("Fitted")

def prepare_submission(model):
    xgtest = xgb.DMatrix(X_test)
    preds = model.predict(xgtest)    
    sub = pd.DataFrame(data = {'listing_id': X_test['listing_id'].ravel()})
    sub['low'] = preds[:, 0]
    sub['medium'] = preds[:, 1]
    sub['high'] = preds[:, 2]
    sub.to_csv("submission.csv", index = False, header = True)

prepare_submission(clf)

In [ ]:
def prepare_submission(model):
    xgtest = xgb.DMatrix(X_test)
    preds = model.predict(xgtest)    
    sub = pd.DataFrame(data = {'listing_id': X_test['listing_id'].ravel()})
    sub['low'] = preds[:, 0]
    sub['medium'] = preds[:, 1]
    sub['high'] = preds[:, 2]
    sub.to_csv("submission.csv", index = False, header = True)

prepare_submission(clf)

In [ ]:
import os
import subprocess as sub
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('../input/images_sample/6811957/') if isfile(join('../input/images_sample/6811957/', f))]
print (onlyfiles)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
img=[]
for i in range (0,5):
    img.append(mpimg.imread('../input/images_sample/6811957/'+onlyfiles[i]))
    plt.imshow(img[i])
    fig = plt.figure()
    a=fig.add_subplot()
    

In [ ]:


X_train.dropna(subset = ['interest_level'])
X_train.shape
print (X_train['interest_level'])


In [ ]:

grouped = X_train.groupby(['interest_level'])
print (grouped.size())
#probability
print (grouped.size()/len(X_train))



In [ ]:
from sklearn.model_selection import train_test_split
X_train["num_photos"] = X_train["photos"].apply(len)
X_train["num_features"] = X_train["features"].apply(len)
X_train["num_description_words"] = X_train["description"].apply(lambda x: len(x.split(" ")))
X_train["created"] = pd.to_datetime(X_train["created"])
X_train["created_year"] = X_train["created"].dt.year
X_train["created_month"] = X_train["created"].dt.month
X_train["created_day"] = X_train["created"].dt.day


In [ ]:

X_test["num_photos"] = X_test["photos"].apply(len)
X_test["num_features"] = X_test["features"].apply(len)
X_test["num_description_words"] = X_test["description"].apply(lambda x: len(x.split(" ")))
X_test["created"] = pd.to_datetime(X_test["created"])
X_test["created_year"] = X_test["created"].dt.year
X_test["created_month"] = X_test["created"].dt.month
X_test["created_day"] = X_test["created"].dt.day



In [ ]:
num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
longitude=X_train["longitude"]
latitude=X_train["latitude"]
listing_id=X_train["listing_id"]
plt.scatter(longitude, latitude)

In [ ]:
plt.scatter(listing_id, latitude)

In [ ]:
plt.scatter(listing_id, longitude)

In [ ]:
X_train3=X_train[X_train["latitude"]==0]
X_train3

In [ ]:
X_train2=X_train[X_train["longitude"]!=0]
X_train2

In [ ]:
plt.scatter(X_train2["longitude"], X_train2["latitude"])

In [ ]:
X_train4=X_train2[X_train2["longitude"]>-110]
plt.scatter(X_train4["longitude"], X_train4["latitude"])

In [ ]:
X_train5=X_train2[X_train2["longitude"]<-110]
X_train5["interest_level"]

In [ ]:
X_train5.T

In [ ]:
#Preprocessing
from wordcloud import WordCloud
text = ''
text_dispadd = ''
text_street = ''
text_desc =''
for ind, row in train.iterrows():
    for feature in row['features']:
        text = " ".join([text, "_".join(feature.strip().split(" "))])
    text_dispadd = " ".join([text_dispadd,"_".join(row['display_address'].strip().split(" "))])
    text_street = " ".join([text_street, row['street_address']])
    text_desc=" ".join([text_desc, row['description']])
text = text.strip()
text_dispadd = text_dispadd.strip()
text_street = text_street.strip()
text_desc = text_desc.strip()

In [ ]:
list(X_train.columns.values)

In [ ]:
target_num_map = {'high':0, 'medium':1, 'low':2}
X_train["label"] = X_train["interest_level"].apply(lambda x: target_num_map[x])
X_train.head()


In [ ]:

y = X_train["label"]

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y, test_size=0.3)

In [ ]:
CATEGORICAL_COLUMNS = ["building_id","description", "display_address", "street_address"]
#CATEGORICAL_COLUMNS = ["building_id","description", "display_address", "features", "street_address"]
CONTINUOUS_COLUMNS = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_photos", "listing_id", 
                      "num_features", "num_description_words", "created_year", "created_month", "created_day"]
LABEL_COLUMN =["label"]
import tensorflow as tf
continuous_cols = {k: tf.constant(X_train[k].values) for k in CONTINUOUS_COLUMNS}
continuous_cols


In [ ]:
X_train["label"].size


In [ ]:
import tensorflow as tf
categorical_cols = {k: tf.SparseTensor(
    indices=[[i, 0] for i in range(X_train[k].size)], 
    values=X_train[k].values, 
    dense_shape = [X_train[k].size , 1]) 
                    for k in CATEGORICAL_COLUMNS}
categorical_cols


In [ ]:
merged = dict(continuous_cols.items() )
merged.update(categorical_cols.items())
merged

In [ ]:
merged2=continuous_cols
merged2.update(categorical_cols)
merged2

In [ ]:
y = X_train["label"]

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y, test_size=0.3)

In [ ]:
def input_fn(df):
    #Creates a dictionary mapping from each continuous feature column name (k) to
    #the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}      
    #Creates a dictionary mapping from each categorical feature column name (k)
    #to the values of that column stored in a tf.SparseTensor.    
    categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)], values=df[k].values, dense_shape=[df[k].size, 1]) for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols=continuous_cols
    feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL_COLUMN].values)
    #Returns the feature columns and the label.
    return feature_cols, label

def train_input_fn():
    return input_fn(X_train2)

def eval_input_fn():
    return input_fn(X_val)

def test_input_fn():
    return input_fn(X_test)

train_input_fn()

In [ ]:
eval_input_fn()

In [ ]:

building_id = tf.contrib.layers.sparse_column_with_hash_bucket("building_id", hash_bucket_size=1000)
description = tf.contrib.layers.sparse_column_with_hash_bucket("description", hash_bucket_size=1000)
display_address = tf.contrib.layers.sparse_column_with_hash_bucket("display_address", hash_bucket_size=1000)
street_address = tf.contrib.layers.sparse_column_with_hash_bucket("street_address", hash_bucket_size=1000)

bathrooms = tf.contrib.layers.real_valued_column("bathrooms")
bedrooms = tf.contrib.layers.real_valued_column("bedrooms")
latitude = tf.contrib.layers.real_valued_column("latitude")
longitude = tf.contrib.layers.real_valued_column("longitude")
price = tf.contrib.layers.real_valued_column("price")
num_photos = tf.contrib.layers.real_valued_column("num_photos")
listing_id = tf.contrib.layers.real_valued_column("listing_id")
num_features = tf.contrib.layers.real_valued_column("num_features")
num_description_words = tf.contrib.layers.real_valued_column("num_description_words")
created_year = tf.contrib.layers.real_valued_column("created_year")
created_month = tf.contrib.layers.real_valued_column("created_month")
created_day = tf.contrib.layers.real_valued_column("created_day")

In [ ]:
daddress_x_saddress = tf.contrib.layers.crossed_column([display_address, street_address], hash_bucket_size=int(1e4))


In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.LinearClassifier(feature_columns=[
  description, street_address, bathrooms, bedrooms, latitude, longitude, price,
  num_photos, num_features, num_description_words, created_year, created_month, created_day],
  model_dir=model_dir)

In [ ]:
m.fit(input_fn=train_input_fn, steps=200)

In [ ]:
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print ("%s: %s" % (key, results[key]))

In [ ]:
results2 = m.evaluate(input_fn=train_input_fn, steps=1)
for key in sorted(results2):
    print ("%s: %s" % (key, results2[key]))

In [ ]:
def input_fn2(df):
    #Creates a dictionary mapping from each continuous feature column name (k) to
    #the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}      
    #Creates a dictionary mapping from each categorical feature column name (k)
    #to the values of that column stored in a tf.SparseTensor.    
    categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)], values=df[k].values, dense_shape=[df[k].size, 1]) for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols=continuous_cols
    feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    #label = tf.constant(df[LABEL_COLUMN].values)
    #Returns the feature columns and the label.
    return feature_cols

def test_input_fn():
    return input_fn2(X_test)

test_input_fn()

In [ ]:
print (m.predict(input_fn=lambda: test_input_fn()))